In [0]:
!pip install tensorflow==1.14

Importujemy biblioteki:

In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from sklearn.decomposition import PCA
from matplotlib import pyplot
import numpy as np


Tworzymy dane wejściowe i wyjściowe:

In [0]:
docs = ['fantasticly done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
labels = np.array([[1.],[1.],[1.],[1.],[1.],[0.],[0.],[0.],[0.],[0.]])


Zamieniemy słowa na losowo wybrane liczby w zakresie do 30

In [0]:
vocab_size = 100
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)


Wyrównujemy wejście, żeby wszystkie były takie same:

In [0]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

Definiujemy model:

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
#model.add(Dense(30, input_shape=(4,)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Trenujemy i oceniamy model:

In [0]:
model.fit(padded_docs, labels, epochs=100, verbose=0)

loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

Prezentujemy embeddingsy na wykresie:

In [0]:
words = ['fantasticly','good','great','nice','excellent','weak','poor']
encoded_word = [one_hot(d, vocab_size) for d in words]

line = to_categorical(encoded_word, num_classes=vocab_size)


weights = model.layers[0].get_weights()[0]

#show embeddings
print(weights.shape)

emb_words = np.dot(line, weights)


pca = PCA(n_components=2)
result = pca.fit_transform(emb_words)

pyplot.scatter(result[:, 0], result[:, 1])

for i, word in enumerate(words):
	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()
